In [1]:
import pyhdb
connection = pyhdb.connect(
    host="",
    port=30015,
    user="",
    password=""
)
cursor = connection.cursor()

In [2]:
import numpy as np
import tensorflow as tf

model_path = '../2-model/model/output_graph_friends.pb'
labels_path = '../2-model/model/output_labels_friends.txt'

In [3]:
def create_graph():
    with tf.gfile.FastGFile(model_path, 'rb') as model_file:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(model_file.read())
        _ = tf.import_graph_def(graph_def, name='')

In [4]:
def run_inference_on_image(image_data):
    answer = None

    create_graph()

    with tf.Session() as tf_session:

        softmax_tensor = tf_session.graph.get_tensor_by_name('final_result:0')
        predictions = tf_session.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions)

        top_k = predictions.argsort()[-5:][::-1]
        label_file = open(labels_path, 'rb')
        lines = label_file.readlines()
        labels = [str(line).replace("\n", "") for line in lines]
        for node_id in top_k:
            predicted_label = labels[node_id]
            score = predictions[node_id]
            print('%s (score = %.5f)' % (predicted_label, score))

        answer = labels[top_k[0]]
        return answer

In [5]:
from flask import Flask
app = Flask(__name__)

import pyhdb
import io
from flask import request
from flask import send_file

from flask import Flask, jsonify, request
from io import BytesIO
import base64
import re
import json

@app.route('/process_image', methods=['post'])
def process_image():
    image_data = re.sub('^data:image/.+;base64,', '', request.form['data'])
    predicted_label = run_inference_on_image(image_data.decode('base64'))
    result = []
    id = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[0]
    name = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[1]
    dob = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[2]
    charge = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[3]
    charge_date = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[4]
    image = cursor.execute('select * from images_db.images_master where lower(name) like (:1)', ["%" + predicted_label + "%"]).fetchone()[5].read()
    result.append(id)
    result.append(name)
    result.append(dob)
    result.append(charge)
    result.append(charge_date)
    result.append(image)
    return jsonify(result)

In [ ]:
# http://localhost:5000/
app.run()

In [11]:
connection.close()